In [1]:
import numpy as np
import cv2

align_line = np.array([[[1361, 1411],
                        [1477, 1245]]])

plane = {"0_pos_rot": [[0.06916698068380356, -0.5518658757209778, -1.5223110914230347],
                       [115.76455577754594, 7.12439650246335, 0]]}

img_path = r"D:\3d-reconstruction\easy-ai-exterior\server\oldui_data\8659525e-8aa6-47c4-a4b1-815289dd473d\input.jpg"
vertices = np.load(r"D:\3d-reconstruction\easy-ai-exterior\server\oldui_data\8659525e-8aa6-47c4-a4b1-815289dd473d\vertices.npy")
vertices.shape

(3000000, 3)

In [2]:
mask = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
mask[:, :] = 0

this_group_mask = mask.copy()

for line in align_line:
    for point in line:
        print(point)
        this_group_mask[ int(point[1]), int(point[0])] = 255

this_group_mask = this_group_mask.reshape(-1,1)

idx_selected_vertices = np.where(this_group_mask==255)[0]
print(idx_selected_vertices)

selected_vertices = vertices[idx_selected_vertices]

point1 = selected_vertices[0]
point2 = selected_vertices[1]

selected_vertices

[1361 1411]
[1477 1245]
[2491477 2823361]


array([[ 0.47336361, -0.49120682, -1.53820431],
       [ 0.32089314, -0.58719361, -1.37735081]])

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.spatial.transform import Rotation as R

# ✅ Step 1: Define Plane Position & Rotation (Three.js uses radians)
plane_position = np.array(plane['0_pos_rot'][0])
plane_rotation = np.radians(plane['0_pos_rot'][1])  # Convert from degrees to radians

# ✅ Step 2: Define the Plane Dimensions
plane_width = 4
plane_height = 4
half_w, half_h = plane_width / 2, plane_height / 2

# Define the 4 corners in local space (before transformation)
corners_local = np.array([
    [-half_w, -half_h, 0],  # Bottom-left
    [ half_w, -half_h, 0],  # Bottom-right
    [ half_w,  half_h, 0],  # Top-right
    [-half_w,  half_h, 0]   # Top-left
])

# Apply rotation using SciPy's Rotation module
rotation_matrix = R.from_euler('XYZ', plane_rotation).as_matrix()
corners_world = np.dot(corners_local, rotation_matrix.T) + plane_position
print("corners_world", corners_world)

# ✅ Step 3: Define the Plane Axes
axis_length = 2
x_axis_local = np.array([axis_length, 0, 0])  # X-axis in local space
y_axis_local = np.array([0, axis_length, 0])  # Y-axis in local space

# Transform axes to world space
x_axis_world = np.dot(x_axis_local, rotation_matrix.T) + plane_position
y_axis_world = np.dot(y_axis_local, rotation_matrix.T) + plane_position

# # ✅ Step 4: Plot the Plane, Edges, and Axes
# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')

# Plot the edges of the plane
edges = [
    (corners_world[0], corners_world[1]),  # Bottom edge
    (corners_world[1], corners_world[2]),  # Right edge
    (corners_world[2], corners_world[3]),  # Top edge
    (corners_world[3], corners_world[0])   # Left edge
]
print("edges", edges)

corners_world [[-1.91539146  0.09409324 -3.43130681]
 [ 2.05372542  0.54087126 -3.21566674]
 [ 2.05372542 -1.19782499  0.38668462]
 [-1.91539146 -1.64460301  0.17104456]]
edges [(array([-1.91539146,  0.09409324, -3.43130681]), array([ 2.05372542,  0.54087126, -3.21566674])), (array([ 2.05372542,  0.54087126, -3.21566674]), array([ 2.05372542, -1.19782499,  0.38668462])), (array([ 2.05372542, -1.19782499,  0.38668462]), array([-1.91539146, -1.64460301,  0.17104456])), (array([-1.91539146, -1.64460301,  0.17104456]), array([-1.91539146,  0.09409324, -3.43130681]))]


In [4]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.spatial.transform import Rotation as R

# ✅ Step 1: Define Plane Position & Rotation
# plane_position = np.array([-0.5716618895530701, 0.44756120443344116, -1.5269027948379517])
# plane_rotation = np.radians([-78.0041637440927, -89.59539193800651, 0])  # Convert from degrees to radians

# ✅ Step 2: Define Two Points
# point1 = np.array([-0.58744991, 0.06091256, -0.89432341])
# point2 = np.array([-0.59039378, -0.3504971, -0.89219105])

# Compute the midpoint of the line
line_midpoint = (point1 + point2) * 0.5

# ✅ Step 4: Compute translation to move the line to the plane's center
translation = plane_position - line_midpoint
moved_point1 = point1 + translation
moved_point2 = point2 + translation

# ✅ Step 6: Calculate angle of the line with the plane's X-axis
line_direction = (moved_point2 - moved_point1) / np.linalg.norm(moved_point2 - moved_point1)  # Normalize
plane_x_axis = np.array([1, 0, 0])  # X-axis in local space

# Rotate the X-axis to align with the plane’s orientation
rotation_matrix = R.from_euler('XYZ', plane_rotation).as_matrix()
plane_x_axis_world = np.dot(rotation_matrix, plane_x_axis)

# Compute the angle using the dot product
angle_rad = np.arccos(np.clip(np.dot(line_direction, plane_x_axis_world), -1.0, 1.0))
angle_deg = np.degrees(angle_rad)

print(f"Angle between the line and plane's X-axis: {angle_deg:.2f} degrees")

Angle between the line and plane's X-axis: 129.41 degrees


In [5]:
plane_rotation

array([2.02047266, 0.12434418, 0.        ])

In [6]:
import numpy as np
from scipy.spatial.transform import Rotation as R

# ✅ Step 1: Define Plane Position & Rotation (Input in Degrees)
plane_rotation_degrees = plane['0_pos_rot'][1]
plane_rotation_radians = np.radians(plane_rotation_degrees)  # Convert to radians

# Compute the transformed X-axis of the plane
plane_x_axis = np.array([1, 0, 0])  # Local X-axis
rotation_matrix = R.from_euler('XYZ', plane_rotation).as_matrix()
plane_x_axis_world = np.dot(rotation_matrix, plane_x_axis)

# Compute the initial angle
angle_rad = np.arccos(np.clip(np.dot(line_direction, plane_x_axis_world), -1.0, 1.0))
angle_deg = np.degrees(angle_rad)

print(f"Initial angle: {angle_deg:.2f} degrees")

# ✅ Step 2: Function to compute the new angle after rotating around Z
def get_final_angle(test_rotation_z):
    test_rotation = plane_rotation.copy()
    test_rotation[2] = test_rotation_z  # Apply test Z-rotation
    test_rotation_matrix = R.from_euler('XYZ', test_rotation).as_matrix()
    test_plane_x_axis = np.dot(test_rotation_matrix, np.array([1, 0, 0]))  # Transformed X-axis
    
    new_angle_rad = np.arccos(np.clip(np.dot(line_direction, test_plane_x_axis), -1.0, 1.0))
    return np.degrees(new_angle_rad)

# ✅ Step 3: Compute both possible Z-rotations
clockwise_rotation_z = plane_rotation[2] + np.radians(-angle_deg)
counter_clockwise_rotation_z = plane_rotation[2] + np.radians(angle_deg)

# ✅ Step 4: Compute final angles
final_angle_clockwise = get_final_angle(clockwise_rotation_z)
final_angle_counter_clockwise = get_final_angle(counter_clockwise_rotation_z)

# ✅ Step 5: Choose the rotation that minimizes the absolute angle
if abs(final_angle_clockwise) < abs(final_angle_counter_clockwise):
    best_rotation_z = clockwise_rotation_z
else:
    best_rotation_z = counter_clockwise_rotation_z

# ✅ Step 7: Convert final plane rotation back to degrees
plane_rotation_radians[2] = best_rotation_z  # Update Z rotation
final_plane_rotation_degrees = np.degrees(plane_rotation_radians)  # Convert back to degrees

# ✅ Step 8: Print final results
print(f"Best rotation Z: {final_plane_rotation_degrees[2]:.2f} degrees")
print(f"New minimized angle after rotation: {min(abs(final_angle_clockwise), abs(final_angle_counter_clockwise)):.2f} degrees")

print(f"Final Plane Rotation (Degrees): X = {final_plane_rotation_degrees[0]:.6f}, "
      f"Y = {final_plane_rotation_degrees[1]:.6f}, "
      f"Z = {final_plane_rotation_degrees[2]:.6f}")


Initial angle: 129.41 degrees
Best rotation Z: 129.41 degrees
New minimized angle after rotation: 0.60 degrees
Final Plane Rotation (Degrees): X = 115.764556, Y = 7.124397, Z = 129.412056
